In [3]:
import numpy as np
import re
import fitz 
import os
from PIL import Image

# **Get image from pdf**

In [4]:
# constants variables
data_path = r".\\raw_data\\Prj_06_CLC_Hoang_Sách Nôm công giáo 1995 - 110 - Tu Nguyen Yeu Ly - Phan II.pdf"
data_HN_output_path = r".\\output\\Khanh_HN_raw_data_2"
data_QN_output_path = r".\\output\\Khanh_QN_raw_data_2"

In [33]:
start_HN_page = 202
end_HN_page = 212

start_QN_page = 32
end_QN_page = 42

In [31]:
def extract_QN_images_from_pdf(pdf_path, output_folder, start_page, end_page):
    pdf_document = fitz.open(pdf_path)


    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    index = 1
    for page_number in range(start_page, end_page + 1):
        page = pdf_document[page_number]
        images = page.get_images(full=True)  

        for img_index, img in enumerate(images):
            xref = img[0]  
            base_image = pdf_document.extract_image(xref)
            image_bytes = base_image["image"]
            image_ext = base_image["ext"]  

            image_filename = f"{index}.{image_ext}"
            index+=1
            image_path = os.path.join(output_folder, image_filename)
            
            with open(image_path, "wb") as img_file:
                img_file.write(image_bytes)

    print(f"Ảnh đã được trích xuất và lưu tại {output_folder}")


In [14]:
def extract_HN_images_from_pdf(pdf_path, output_folder, start_page, end_page):
    pdf_document = fitz.open(pdf_path)

    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    index = 1
    for page_number in range(end_page, start_page - 1, -1):
        page = pdf_document[page_number]
        images = page.get_images(full=True)  

        for img_index, img in enumerate(images):
            xref = img[0]  
            base_image = pdf_document.extract_image(xref)
            image_bytes = base_image["image"]
            image_ext = base_image["ext"]  

            image_filename = f"{index}.{image_ext}"
            index+=1
            image_path = os.path.join(output_folder, image_filename)
            
            with open(image_path, "wb") as img_file:
                img_file.write(image_bytes)

    print(f"Ảnh đã được trích xuất và lưu tại {output_folder}")

In [34]:
extract_HN_images_from_pdf(data_path, data_HN_output_path, start_HN_page, end_HN_page)
extract_QN_images_from_pdf(data_path, data_QN_output_path, start_QN_page, end_QN_page)

Ảnh đã được trích xuất và lưu tại .\\output\\Khanh_HN_raw_data_2
Ảnh đã được trích xuất và lưu tại .\\output\\Khanh_QN_raw_data_2


# **Change Image**

In [5]:
raw_image_HN_folder_path = r".\\output\\Khanh_HN_raw_data_2"
raw_image_QN_folder_path = r".\\output\\Khanh_QN_raw_data_2"
Page_image_HN_output_path = r".\\output\\Khanh_HN_pages_2"
Page_image_QN_output_path = r".\\output\\Khanh_QN_pages_2"
imageHN_name = r"Khanh_TNYL_P2_HN_page"
imageQN_name = r"Khanh_TNYL_P2_QN_page_"

In [7]:
unrespected_image_HN_index = {1 : False, 38 : False, 39 : False, 170 : False}
unrespected_image_QN_index = {30 : False, 31 : False}

unrespected_image_HN_index = {}
unrespected_image_QN_index = {}


In [1]:
def count_files_in_folder(folder_path, extensions=('.png', '.jpg', '.jpeg')):
    return len([f for f in os.listdir(folder_path) if f.lower().endswith(extensions)])

def Data_process(raw_image_folder_path, Page_image_output_path, image_name, IsHN=True):
    if not os.path.exists(Page_image_output_path):
        os.makedirs(Page_image_output_path)

    index = 1
    count_image = count_files_in_folder(raw_image_folder_path)
    count = 0
    for i in range(1, count_image + 1):
        _, extension = os.path.splitext(f'{i}.png')
        file_path = os.path.join(raw_image_folder_path, f'{i}.png')

        with Image.open(file_path) as image:
                # Xoay ảnh 90 độ

            # image = image.rotate(90, expand=True)
            # image.save(file_path)
            width, height = image.size

            # Cắt ảnh ban đầu
            if IsHN:
                croped_image = image.crop((450, 150, width - 300, height))
                mid_x = (croped_image.width // 2) - 50
                left_half = croped_image.crop((0, 0, mid_x, croped_image.height))
                right_half = croped_image.crop((mid_x, 0, croped_image.width , croped_image.height))
            else:
                croped_image = image.crop((200, 150, width , height - 420))
                mid_x = (croped_image.width // 2) - 20
                left_half = croped_image.crop((0, 0, mid_x, croped_image.height))
                right_half = croped_image.crop((mid_x, 0, croped_image.width - 100, croped_image.height))

            

            # Lưu 2 phần ảnh
            if(IsHN):
                flag = True
                for item in unrespected_image_HN_index.keys():
                    if(index == item - count and unrespected_image_HN_index[item] == False):
                        unrespected_image_HN_index[item] = True
                        count += 1
                        flag = False
                        print(f"khong in {index}")
                        break
                if(flag):
                    right_image_path = os.path.join(Page_image_output_path, f"{image_name}{str(index).zfill(3)}{extension}")
                    index += 1
                    right_half.save(right_image_path)

                flag = True
                for item in unrespected_image_HN_index.keys():
                    if(index == item - count and unrespected_image_HN_index[item] == False):
                        unrespected_image_HN_index[item] = True
                        print(f"khong in {index}")
                        count += 1
                        flag = False
                        break
                if(flag):
                    left_image_path = os.path.join(Page_image_output_path, f"{image_name}{str(index).zfill(3)}{extension}")
                    index += 1
                    left_half.save(left_image_path)
            else:
                flag = True
                for item in unrespected_image_QN_index.keys():
                    if(index == item - count and unrespected_image_QN_index[item] == False):
                        unrespected_image_HN_index[item] = True
                        count += 1
                        flag = False
                        break
                if(flag):
                    left_image_path = os.path.join(Page_image_output_path, f"{image_name}{index}{extension}")
                    index += 1
                    left_half.save(left_image_path)

                flag = True
                for item in unrespected_image_QN_index.keys():
                    if(index == item - count and unrespected_image_QN_index[item] == False):
                        unrespected_image_HN_index[item] = True
                        count += 1
                        flag = False
                        break
                if(flag):
                    right_image_path = os.path.join(Page_image_output_path, f"{image_name}{index}{extension}")
                    index += 1
                    right_half.save(right_image_path)

    print("Done")


In [8]:
Data_process(raw_image_HN_folder_path, Page_image_HN_output_path, imageHN_name)
# Data_process(raw_image_QN_folder_path, Page_image_QN_output_path, imageQN_name, IsHN=False)

Done
